# Is it Huggable?
*Classify objects as huggable or not.*

This notebook has steps to make the model.

Just want to play? Use directly on the [website](https://daspartho.github.io/is-it-huggable).

### Install required libraries

In [ ]:
!pip install -Uqq fastai duckduckgo_search

### Import modules required

In [2]:
from duckduckgo_search import ddg_images
from fastcore.all import *
from fastdownload import download_url
from fastai.vision.all import *

### Use DuckDuckGo to search for images of examples of the two groups

In [ ]:
def search_images(term, max_images=50):
    print(f"Searching for '{term}'")
    return L(ddg_images(term, max_results=max_images)).itemgot('image')

path = Path('huggable_or_not')

# examples of both groups
categories={
   'huggable':['plushie', 'pillow' , 'ballon', 'dog'], 
   'not huggable':['chainsaw', 'sword', 'cactus', 'barbwire']
   }

for category in categories:
    dest = (path/category)
    dest.mkdir(exist_ok=True, parents=True)
    for example in categories[category]:
        download_images(dest, urls=search_images(f'{example} photo'))
        resize_images(path/category, max_size=400, dest=path/category)

### Remove photos that didn't download correctly.

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

### Preparing the data for training

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), # inputs to our model are images, and the outputs are categories
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42), # Split the data into training and validation sets randomly, using 20% of the data for the validation set
    get_y=parent_label, # The labels is the name of the parent of each file
    item_tfms=RandomResizedCrop(224, min_scale=0.3), # picks a random scaled crop of an image and resize it to 224x224 pixels
    batch_tfms=aug_transforms() # applies augmentations to an entire batch
).dataloaders(path, bs=32)

dls.show_batch()

### Fine-tune a pretrained neural network to recognise these two groups

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(3)

### Show predictions the model made on images in validation set

In [ ]:
learn.show_results()

### Download an image from internet for trying the model

In [ ]:
term='snowman' # change the search term
download_url(search_images(term, max_images=1)[0], 'test.jpg', show_progress=False)
Image.open('test.jpg').to_thumb(256,256)

### Trying the model on the downloaded image

In [ ]:
predict,n,prob = learn.predict(PILImage.create('test.jpg'))
print(f"It's {predict}!")
perc = prob[n]*100
print(f"I'm {perc:.02f}% confident.")

### Export the model

In [40]:
learn.export('model.pkl')